<a href="https://colab.research.google.com/github/Tonoyama/DL-Study/blob/master/Answer_original_data_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 自作データセットを CNN(Convolutional Neural Network) で分析する

今まで、MNIST や Fasion MNIST といった有名なデータセットを使ってきたが、研究や趣味などで自分の好きなデータセットを分類したい場合は多い。

今回は、自作データセットを読み込み CNN で分類を行う。

## `OpenCV` の使い方

`OpenCV` は BSDライセンスの画像処理ライブラリ。Python と C++ で使える。

`OpenCV` 以外にも `Pillow` や `Numpy` も画像処理に使う。

標準画像データベースSIDBA(Standard Image Data-BAse)が公開している有名な画像を使う。

http://www.ess.ic.kanagawa-it.ac.jp/app_images_j.html よりマンドリル(サルの一種)の画像を `curl` で取ってくる。

`-o` オプションで `Mandrill.jpg` という名前にする。

`colab` ファイル内に `Mandrill.jpg` が増えている。

In [ ]:
!curl http://www.ess.ic.kanagawa-it.ac.jp/std_img/colorimage/Mandrill.jpg -o Mandrill.jpg


`opencv 画像 表示` で検索し、画像(`Mandrill.jpg`)を**グレースケール**で表示する。

このとき、`colab` 特有のエラーが出たりする場合もあるので、その際は、エラーログを調べる。

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

img_name = "Mandrill.jpg"

img = cv2.imread(img_name, 0)
cv2_imshow(img)

In [ ]:
img = cv2.imread(img_name)
cv2_imshow(img)

`opencv python` で検索すると様々な機能を紹介するチュートリアルを見つけられる。

遊んでみると結構楽しい

カラー画像のヒストグラムをプロットしてみると、赤と青の輝度が大きい事が分かる

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

img = cv2.imread(img_name)
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv2.calcHist([img],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

エッジ検出してみる。

`min_val` を 100、`max_val` を 200 ぐらいにするといい感じ

In [ ]:
import cv2

gray_img = cv2.imread(img_name, 0)
canny_img = cv2.Canny(gray_img, 100, 200)
cv2_imshow(canny_img)

## 犬と猫を分類する

画像を表示してみる。

### データセットを用意する

今回は 手作業でも `curl` でも出来るようにしている。


#### 手作業の場合

下記の URL から zip ファイルをダウンロードし、Google ドライブにファイルをアップロードする。

https://www.microsoft.com/en-us/download/details.aspx?id=54765

`drive.mount('/content/drive')` でドライブをマウントし、`MyDrive`内にある zip ファイルを `unzip` する。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### curl の場合

`-o` オプションで `cat_dog.zip` という名前にする。

あとは、`unzip`すれば良い。

In [ ]:
!curl https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip -o cat_dog.zip

In [ ]:
!unzip /content/cat_dog.zip

### データセットを確認する

画像の命名規則は `数字` + `.jpg` になっていて、サイズは不揃い。

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread
folder = "/content/PetImages/Dog/"

for i in range(9):
	pyplot.subplot(330 + 1 + i)
	filename = folder + str(i) + '.jpg'
	image = imread(filename)
	pyplot.imshow(image)
pyplot.show()

自作データセットでは、画像サイズが揃っていない場合が多いため、全て揃えてやる必要がある。

拡張子が揃っていない場合、pillow で変換してやると良い。

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread
folder = "/content/PetImages/Cat/"

for i in range(9):
	pyplot.subplot(330 + 1 + i)
	filename = folder + str(i) + '.jpg'
	image = imread(filename)
	pyplot.imshow(image)
pyplot.show()

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import random
import numpy as np
from tqdm.notebook import tqdm


DATADIR = "./PetImages" # 全データセットが含まれるフォルダ名

CATEGORIES = ["Dog", "Cat"] # 各フォルダ名(カテゴリ名)を指定

IMG_SIZE = 50 # 画像サイズを指定

training_data = [] # 画像データ、ラベル情報 を入れるための空配列


def create_training_data():
    # enumerate() を使うと　インデックス と 要素 を同時に取得できる
    for class_num, category in enumerate(CATEGORIES):
        path = os.path.join(DATADIR, category)
        for image_name in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path, image_name),)  # 画像読み込み
                img_resize_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # 画像のリサイズ
                training_data.append([img_resize_array, class_num])  # 画像データ、ラベル情報を追加
            except Exception as e:
                pass

create_training_data()


random.shuffle(training_data)  # データをシャッフル


X = []  # 画像データ
y = []  # ラベル情報

# データセット作成
# 画像(feature), ラベル情報(label)
for feature, label in training_data:
    X.append(feature)
    y.append(label)

# numpy配列に変換
X = np.array(X)
y = np.array(y)

ラベルとデータを可視化

In [ ]:
# データセットの確認
for i in range(0, 4):
    print("学習データのラベル：", y[i])
    plt.subplot(2, 2, i+1)
    plt.axis('off')
    plt.title(label = 'Dog' if y[i] == 0 else 'Cat')
    img_array = cv2.cvtColor(X[i], cv2.COLOR_BGR2RGB)
    plt.imshow(img_array)

plt.show()

`shape` でデータの次元数を確認する

50 x 50 の RGB であることがわかる

In [ ]:
print(X.shape)
print(y.shape)

### ホールドアウト法でデータセットを分割

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100,
                                                    stratify=y)

X_train, X_test, y_train, y_test の次元を確認する

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

`BGR` になっているので `RGB` に変更する。

In [ ]:
fig = plt.figure(figsize=(9, 15))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.5, hspace=0.05,
                    wspace=0.05)

for i in range(9):
    ax = fig.add_subplot(1, 9, i + 1, xticks=[], yticks=[])
    img_array = cv2.cvtColor(X_train[i], cv2.COLOR_BGR2RGB)
    ax.imshow(img_array)

`X_train`, `X_test` は画素値を `255` で割ることで `0` ~ `1` に正規化する。

`y_train`, `y_test` は正解ラベルを One-hot エンコーディングする。keras を使うと簡単。

In [ ]:
from tensorflow.keras.utils import to_categorical

X_train = X_train.reshape((X_train.shape[0], 50, 50, 3)) / 255
X_test = X_test.reshape((X_test.shape[0], 50, 50, 3)) / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train, X_test, y_train, y_test の次元を確認すると変わっている点がある。

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

### モデル定義

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard

model = Sequential()

model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(50, 50, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(
    loss=keras.losses.binary_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

モデルを可視化する

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

Keras の実装は下記 URL をみると様々な実装があって良い。

https://keras.io/examples

### 学習

In [ ]:
tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)

model.fit(x=X_train, y=y_train, batch_size=128, epochs=10, verbose=1,
          validation_data=(X_test, y_test), callbacks=[tf_callback])

model.evaluate(X_test,  y_test, verbose=2)

TensorBoard で結果を可視化する。

In [ ]:
%tensorboard --logdir logs